In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.formula.api as smf

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 200

In [16]:
def print_def(variable_name):
    """
    Pass in a variable name as a string, print its defintion.
    Definitions included with the original dataset. 
    Only works with ORIGINAL feature names.
    """
    
    try:
        if  not ("definitions_" in globals()):
            try:
                global definitions_
                definitions_ = pd.read_csv("LC_definitions.csv")       
            except:
                print(f"Could not find \"LC_definitions.csv\" in {pwd}.")
        print(variable_name, "-", definitions_.loc[definitions_.LoanStatNew == variable_name, "Description"].iloc[0])
    except:
        print("New feature, not in original dictionary.")

In [17]:
# Either pass in a list of columns to use via logit_cols_ 
# or pass in a list of columns to drop via logit_drop_
def smf_logit(df_, y_, logit_cols_=None, logit_drop_=None):
    if logit_drop_:
        sm_df = df_.drop(labels=logit_drop_, axis=1).dropna()
    elif logit_cols_:
        sm_df = df_.loc[:, logit_cols_+[y_]].copy().dropna()   
    else:
        sm_df = df_.copy().dropna()
        
    formula = y_ + " ~ "
    for column in sm_df.drop(labels=[y_], axis=1).columns:
        formula += column + " + "
    formula = formula[:-3]
    
    print(f"{formula}")
    sm_model = smf.logit(formula, data=sm_df).fit()
    print(sm_model.summary())

In [21]:
df = pd.read_csv("lending_club_sample_preprocessed.csv")
df_biased = pd.read_csv("lending_club_biased_preprocessed.csv")

In [19]:
use_cols = ["last_fico_range_high", "last_pymnt_amnt", "total_rec_prncp", 
            "funded_amnt", "issue_date_year", "installment", 
            "total_pymnt_inv", "total_pymnt"]

smf_logit(df, "target", logit_cols_=use_cols)

target ~ last_fico_range_high + last_pymnt_amnt + total_rec_prncp + funded_amnt + issue_date_year + installment + total_pymnt_inv + total_pymnt
Optimization terminated successfully.
         Current function value: 0.109749
         Iterations 13
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                60000
Model:                          Logit   Df Residuals:                    59991
Method:                           MLE   Df Model:                            8
Date:                Wed, 09 Aug 2023   Pseudo R-squ.:                  0.7090
Time:                        10:04:32   Log-Likelihood:                -6584.9
converged:                       True   LL-Null:                       -22628.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------